In [ ]:
#!usr/bin/env python3
import pandas as pd
from pathlib import Path
import praw
from tqdm import tqdm
import pandas as pd
import yfinance as yf
import zstandard
from pathlib import Path

In [2]:
root = Path.cwd()
compressed_data_root = root / 'posts'
reddit_csv_path = root.parent / '10y_reddit_data.csv'
stock_csv_path = root.parent / '10y_stock_data.csv'

years=10

subreddits = ["stocks", "StockMarket", "investing", "wallstreetbets", "options", "trading"]

stock_dict = {
    'nvidia': ['nvda', 'nvidia'],
    'tesla': ['tsla', 'tesla'],
    'apple': ['aapl', 'apple'],
    'amazon': ['amzn', 'amazon'],
    'microsoft': ['msft', 'microsoft'],
    'google': ['googl', 'google', 'alphabet']
}

tickers = ['nvda', 'tsla', 'aapl', 'amzn', 'msft', 'googl']

praw_api = praw.Reddit(
            client_id="5uFqCBUPadVnIxKHG0hnhw",
            client_secret="LDFyai0bjEAkEQqo5joU7PjSjtq2eQ",
            user_agent="TeslaScraper:v1.0 (by u/RecognitionSame5433)"
        )

In [3]:
df = pd.read_json(compressed_data_root/'investing_submissions.zst', lines=True, compression='zstd')

In [4]:
df.columns

Index(['archived', 'author', 'author_flair_background_color',
       'author_flair_css_class', 'author_flair_richtext', 'author_flair_text',
       'author_flair_text_color', 'author_flair_type', 'brand_safe',
       'can_gild',
       ...
       'top_awarded_type', 'event_end', 'event_is_live', 'event_start',
       'url_overridden_by_dest', 'call_to_action', 'author_is_blocked',
       'updated_on', 'previous_selftext', '_meta'],
      dtype='object', length=131)

In [5]:
df.head()

,archived,author,author_flair_background_color,author_flair_css_class,author_flair_richtext,author_flair_text,author_flair_text_color,author_flair_type,brand_safe,can_gild,...,top_awarded_type,event_end,event_is_live,event_start,url_overridden_by_dest,call_to_action,author_is_blocked,updated_on,previous_selftext,_meta
0,1.0,yuckatan,None,None,[],None,None,text,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,Escafane,None,None,[],None,None,text,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,Escafane,None,None,[],None,None,text,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,Escafane,None,None,[],None,None,text,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,Escafane,None,None,[],None,None,text,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
columns = ['id', 'created_utc', 'author', 'title', 'selftext', 'score', 'num_comments']

In [16]:
for path in compressed_data_root.rglob('*.zst'):
    print(path)

/home/stephen/CS560-Group2/data_processing/posts/options_submissions.zst
/home/stephen/CS560-Group2/data_processing/posts/wallstreetbets__submissions.zst
/home/stephen/CS560-Group2/data_processing/posts/stocks_submissions.zst
/home/stephen/CS560-Group2/data_processing/posts/Trading_submissions.zst
/home/stephen/CS560-Group2/data_processing/posts/investing_submissions.zst
/home/stephen/CS560-Group2/data_processing/posts/StockMarket_submissions.zst
/home/stephen/CS560-Group2/data_processing/posts/wallstreetbets_submissions.zst


In [17]:
df.created_utc

0         1205583687
1         1205929080
2         1205929704
3         1205930985
4         1205931725
             ...    
399033    1735682544
399034    1735684012
399035    1735685136
399036    1735688356
399037    1735689406
Name: created_utc, Length: 399038, dtype: int64

In [20]:
ts = pd.to_datetime(df.created_utc[399037], unit='s')

1